In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    accuracy_score, f1_score, confusion_matrix
)

# PARTE 1 - EX 1: REGRESSÃO (APPLIANCES ENERGY PREDICTION)

In [2]:
# Prevendo o consumo de energia
df_energy = pd.read_csv('/content/energydata_complete.csv')
df_energy['date'] = pd.to_datetime(df_energy['date'])
df_energy['hour'] = df_energy['date'].dt.hour
df_energy['day_of_week'] = df_energy['date'].dt.dayofweek

X = df_energy.drop(columns=['date', 'Appliances', 'lights'])
y = df_energy['Appliances']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Testando e avaliando o desempenho dos modelos

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models_energy = {
    '''Regressão Linear''': LinearRegression(),
    '''Árvore de Regressão''': DecisionTreeRegressor(random_state=42),
    '''Random Forest''': RandomForestRegressor(n_estimators=100, random_state=42)
}

results_energy = {}
predictions_energy = {}

for name, model in models_energy.items():
    if name == '''Regressão Linear''':
        model.fit(X_train_scaled, y_train)
        pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

    predictions_energy[name] = pred
    results_energy[name] = [
        r2_score(y_test, pred),
        np.sqrt(mean_squared_error(y_test, pred)),
        mean_absolute_error(y_test, pred)
    ]

# Comparando os resultados

results_df_energy = pd.DataFrame(results_energy, index=['R²', 'RMSE', 'MAE']).T
print(''' Resultados: ''')
print(results_df_energy)

best_model_name = results_df_energy['R²'].idxmax()
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test, y=predictions_energy[best_model_name], alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red',
linestyle='--', lw=2)
plt.title(f'Valores Reais vs Previstos - {best_model_name}')
plt.xlabel('Valores Reais (Wh)')
plt.ylabel('Valores Previstos (Wh)')
plt.grid(True)
plt.show()

print('''\n''')
best_rmse_model = results_df_energy['RMSE'].idxmin()
print(f'''O modelo com o maior R² ({results_df_energy.loc[best_model_name, 'R²']
:.4f}) e menor RMSE ({results_df_energy.loc[best_rmse_model, 'RMSE']:.2f} Wh) é
o '{best_model_name}'.''')
print('''O Random Forest é melhor porque as relações entre as variáveis
ambientais e o consumo não são totalmente lineares, e o modelo captura melhor
estas relações, já que é baseado em árvores.''')

FileNotFoundError: [Errno 2] No such file or directory: '/content/energydata_complete.csv'

# PARTE 1 - EX 2: CLASSIFICAÇÃO (SMART GRID STABILITY)

In [ ]:
# Classificando a estabilidade da rede

df_grid = pd.read_csv('/content/smart_grid_stability_augmented.csv')
df_grid = df_grid.drop('stab', axis=1)

le = LabelEncoder()
df_grid['stabf'] = le.fit_transform(df_grid['stabf'])

X_grid = df_grid.drop('stabf', axis=1)
y_grid = df_grid['stabf']

X_train_grid, X_test_grid, y_train_grid, y_test_grid = train_test_split(
    X_grid, y_grid, test_size=0.3, random_state=42, stratify=y_grid
)

# Treinando e avaliando o desempenho dos modelos

scaler_grid = StandardScaler()
X_train_grid_scaled = scaler_grid.fit_transform(X_train_grid)
X_test_grid_scaled = scaler_grid.transform(X_test_grid)

models_grid = {
    '''Árvore de Decisão''': DecisionTreeClassifier(random_state=42),
    '''KNN''': KNeighborsClassifier(),
    '''Regressão Logística''': LogisticRegression(random_state=42)
}

results_grid = {}

for name, model in models_grid.items():
    if name == '''Árvore de Decisão''':
        model.fit(X_train_grid, y_train_grid)
        predictions = model.predict(X_test_grid)
    else:
        model.fit(X_train_grid_scaled, y_train_grid)
        predictions = model.predict(X_test_grid_scaled)

    results_grid[name] = {
        'Acurácia': accuracy_score(y_test_grid, predictions),
        'F1-Score': f1_score(y_test_grid, predictions),
        'Matriz de Confusão': confusion_matrix(y_test_grid, predictions)
    }


# Comparando os resultados

results_df_grid = pd.DataFrame({n: {'Acurácia': r['Acurácia'], 'F1-Score': r['F1-Score']} for n, r in results_grid.items()}).T
print(''' Resultados: ''')
print(results_df_grid)

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Matrizes de Confusão (0=Estável, 1=Instável)', fontsize=16)
for i, (name, res) in enumerate(results_grid.items()):
    sns.heatmap(res['Matriz de Confusão'], annot=True, fmt='d', cmap='Blues', ax=axes[i], cbar=False)
    axes[i].set_title(name)
    axes[i].set_xlabel('Previsto')
    axes[i].set_ylabel('Verdadeiro')
plt.show()

min_fn = float('inf')
best_model_fn = ''
for name, res in results_grid.items():
    fn = res['Matriz de Confusão'][1, 0]
    if fn < min_fn:
        min_fn = fn
        best_model_fn = name

print(f'''O modelo '{best_model_fn}' é o mais confiável para detectar
instabilidade, porque apresentou o menor número de Falsos Negativos ({min_fn}).''')

# PARTE 2 - EX 1: CLASSIFICAÇÃO (SOLAR)

In [ ]:
#  Classificando períodos em Alta e Baixa Radiação

df_solar = pd.read_csv('/content/SolarPrediction.csv')


radiation_median = df_solar['Radiation'].median()
df_solar['RadiationLevel'] = (df_solar['Radiation'] > radiation_median).astype(int)

X_solar = df_solar.drop(columns=[
    'UNIXTime', 'Data', 'Time', 'Radiation', 'RadiationLevel',
    'TimeSunRise', 'TimeSunSet'
])
y_solar = df_solar['RadiationLevel']

# Separação 70/30
X_train_solar, X_test_solar, y_train_solar, y_test_solar = train_test_split(
    X_solar, y_solar, test_size=0.3, random_state=42, stratify=y_solar
)

#  Comparando o desempenho de três algoritmos do Scikit-learn

scaler_solar = StandardScaler()
X_train_solar_scaled = scaler_solar.fit_transform(X_train_solar)
X_test_solar_scaled = scaler_solar.transform(X_test_solar)

models_solar = {
    '''Árvore de Decisão''': DecisionTreeClassifier(random_state=42),
    '''Random Forest''': RandomForestClassifier(random_state=42),
    '''Support Vector Machine (SVM)''': SVC(random_state=42)
}

results_solar = {}

for name, model in models_solar.items():
    if name == 'Support Vector Machine (SVM)':
        model.fit(X_train_solar_scaled, y_train_solar)
        predictions = model.predict(X_test_solar_scaled)
    else:
        model.fit(X_train_solar, y_train_solar)
        predictions = model.predict(X_test_solar)

    results_solar[name] = {
        'Acurácia': accuracy_score(y_test_solar, predictions),
        'Matriz de Confusão': confusion_matrix(y_test_solar, predictions)
    }

#  Avaliando com acurácia e matriz de confusão

results_df_solar = pd.DataFrame({n: {'Acurácia': r['Acurácia']} for n, r in results_solar.items()}).T
print(''' Resultados: ''')
print(results_df_solar)

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('Matrizes de Confusão (Solar - 0=Baixa Radiação, 1=Alta Radiação)', fontsize=16)
for i, (name, res) in enumerate(results_solar.items()):
    sns.heatmap(res['Matriz de Confusão'], annot=True, fmt='d', cmap='Greens', ax=axes[i], cbar=False)
    axes[i].set_title(name)
    axes[i].set_xlabel('Previsto')
    axes[i].set_ylabel('Verdadeiro')
plt.show()

best_acc_model = results_df_solar['Acurácia'].idxmax()
print(f'''O modelo com a maior acurácia é o '{best_acc_model}'
({results_df_solar.loc[best_acc_model, 'Acurácia']:.4f}).''')
print('''As matrizes de confusão mostram que o Random Forest e o SVM têm alto
 desempenho, com baixas classificações incorretas.''')

# PARTE 2 - EX 2: REGRESSÃO (EÓLICA)

In [ ]:
#  Treinando modelos de regressão para prever a potência gerada

df_wind = pd.read_csv('/content/Wind_Turbine_Scada_Dataset.csv')
df_wind.columns = ['DateTime', 'ActivePower_kW', 'WindSpeed_ms', 'TheoreticalPower_KWH', 'WindDirection']
df_wind = df_wind[df_wind['ActivePower_kW'] >= 0]

X_wind = df_wind[['WindSpeed_ms', 'TheoreticalPower_KWH', 'WindDirection']]
y_wind = df_wind['ActivePower_kW']

# Separação 80/20
X_train_wind, X_test_wind, y_train_wind, y_test_wind = train_test_split(
    X_wind, y_wind, test_size=0.2, random_state=42
)

#  Comparando o desempenho de três algoritmos do Scikit-learn

scaler_wind = StandardScaler()
X_train_wind_scaled = scaler_wind.fit_transform(X_train_wind)
X_test_wind_scaled = scaler_wind.transform(X_test_wind)

models_wind = {
    '''Regressão Linear''': LinearRegression(),
    '''Regressão de Árvores''': DecisionTreeRegressor(random_state=42),
    '''Random Forest Regressor''': RandomForestRegressor(n_estimators=100, random_state=42)
}

results_wind = {}
predictions_wind = {}

for name, model in models_wind.items():
    if name == 'Regressão Linear':
        model.fit(X_train_wind_scaled, y_train_wind)
        pred = model.predict(X_test_wind_scaled)
    else:
        model.fit(X_train_wind, y_train_wind)
        pred = model.predict(X_test_wind)

    predictions_wind[name] = pred
    results_wind[name] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_wind, pred)),
        'R²': r2_score(y_test_wind, pred)
    }

#  Avaliando com RMSE e R2

results_df_wind = pd.DataFrame(results_wind).T
print(''' Resultados: ''')
print(results_df_wind)

best_model_name_wind = results_df_wind['R²'].idxmax()
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test_wind, y=predictions_wind[best_model_name_wind], alpha=0.6)
plt.plot([min(y_test_wind), max(y_test_wind)], [min(y_test_wind), max(y_test_wind)], color='red', linestyle='--', lw=2)
plt.title(f'Valores Reais vs. Previstos - {best_model_name_wind}')
plt.xlabel('Potência Real (kW)')
plt.ylabel('Potência Prevista (kW)')
plt.grid(True)
plt.show()

print('''\n''')

best_rmse_model_wind = results_df_wind['RMSE'].idxmin()
print(f'''A {best_model_name_wind} foi o modelo com melhor desempenho,
 com maior R² ({results_df_wind.loc[best_model_name_wind, 'R²']:.4f})
  e menor RMSE ({results_df_wind.loc[best_rmse_model_wind, 'RMSE']:.2f} kW).''')
print('''O gráfico de dispersão confirma o bom ajuste,
 com pontos seguindo a linha diagonal que representa a melhor previsão''')